[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RonPlusSign/AnomalySegmentation/blob/chris-1/Project6.ipynb)

# **Anomaly Segmentation Project 6**
##*Andrea Delli, Christian Dellisanti, Giorgia Modi*

###**Dataset Preparation**

In [3]:
'''
SMIYC RA-21
if not os.path.isfile('/content/dataset_AnomalyTrack.zip'):
  !wget https://zenodo.org/record/5270237/files/dataset_AnomalyTrack.zip
if not os.path.isdir('/content/dataset_AnomalyTrack'):
  !unzip ./dataset_AnomalyTrack.zip
#SMIYC RO-21
if not os.path.isfile('/content/dataset_ObstacleTrack.zip'):
  !wget https://zenodo.org/record/5281633/files/dataset_ObstacleTrack.zip
if not os.path.isdir('/content/dataset_ObstacleTrack'):
  !unzip ./dataset_ObstacleTrack.zip

#FS L&F
if not os.path.isfile('/content/fishyscapes_lostandfound.zip'):
  !wget http://robotics.ethz.ch/~asl-datasets/Fishyscapes/fishyscapes_lostandfound.zip
if not os.path.isdir('/content/fishyscapes_lostandfound'):
  !unzip ./fishyscapes_lostandfound.zip -d fishyscapes_lostandfound
'''

import sys, os
if not os.path.isfile('/content/Validation_Dataset.zip'):
  !gdown 12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta
if not os.path.isdir('/content/Validation_Dataset'):
  !unzip Validation_Dataset.zip


['7.png', '17.png', '51.png', '11.png', '78.png', '54.png', '36.png', '22.png', '99.png', '58.png', '76.png', '81.png', '30.png', '55.png', '43.png', '8.png', '26.png', '14.png', '49.png', '33.png', '53.png', '79.png', '96.png', '62.png', '61.png', '10.png', '67.png', '68.png', '57.png', '87.png', '45.png', '38.png', '34.png', '3.png', '94.png', '35.png', '86.png', '46.png', '84.png', '29.png', '25.png', '27.png', '16.png', '98.png', '39.png', '74.png', '0.png', '89.png', '64.png', '66.png', '72.png', '20.png', '4.png', '85.png', '1.png', '40.png', '5.png', '47.png', '80.png', '37.png', '70.png', '97.png', '50.png', '90.png', '95.png', '31.png', '82.png', '93.png', '92.png', '13.png', '42.png', '69.png', '52.png', '59.png', '75.png', '77.png', '21.png', '63.png', '9.png', '60.png', '2.png', '73.png', '18.png', '41.png', '23.png', '44.png', '56.png', '32.png', '24.png', '12.png', '19.png', '71.png', '28.png', '88.png', '83.png', '6.png', '91.png', '48.png', '65.png', '15.png']


**DATASET MANAGEMENT**

In [ ]:
from torch.utils.data import Dataset
import os

from PIL import Image
import sys


def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(f"/content/Validation_Dataset/{path}", 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')


# Define the Dataset class
class PACSDataset(Dataset):
    def __init__(self, root= "/content/Validation_Dataset", source_domain="FS_LostFound_full" , dataset_transform=None):
      super(PACSDataset, self).__init__( )

      self.dataset_transform = dataset_transform

      self.root=f"{root}/{source_domain}"
      self.data   = os.listdir(f"{root}/{source_domain}/images")
      #self.labels = os.listdir(f"{root}/{source_domain}/labels_masks")



    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

      image, label = pil_loader(f"images/{self.data[index]}"), pil_loader(f"labels_masks/{self.data[index]}")

      # Applies preprocessing when accessing the image
      if self.dataset_transform is not None:
          image = self.dataset_transform(image)

      return image, label




